<center>
<img src="https://raw.githubusercontent.com/FUlyankin/ekenam_grand_research/master/images/cover.png">
</center>


# <center> Иканам гранд рисёрч </center>
## <center>  Часть первая: собираем данные про иканам </center> 


Проект **Иканам гранд рисёрч** реализуется [Иканам стьюдентс коммьюнити,](https://vk.com/ikanam)
в частности [вот этим парнем по имени Филипп.](https://vk.com/ppilif) Если вы нашли ошибку или у вас есть предложения, замечания, деньги, слава или женщины, можно ему написать. Весь говнокод, использованный в исследовании распространяется по лицензии [Creative Commons CC BY-NC-SA 4.0.](https://creativecommons.org/licenses/by-nc-sa/4.0/) Его можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу рисёрча.](https://github.com/FUlyankin/ekenam_grand_research)



# Сбор данных. 

Предыдущий блокнот:  "Как забрать из вк любые данные" 

Следующий блокнот: "Пока что хз" 

В этом блокноте мы с вами попытаемся пройти через одну из самых сложных частей любого исследования: через сбор необходимых данных. Нам предстоит ворваться на все странички вкМы в этом блокноте ворвёмся на контактовские странички всех ребят с эконома и вероломно ограбить их. Как вы уже догадались, нам в этом очень сильно поможет контактовский API. 

По ходу наших сборов мы всегда будем сохранять данные в двух видах: 

1. Грязные данные (прямо в том виде, в каком их отдал нам контакт)
2. Очищенные красивые таблички, которые можно будет впоследствии использовать для оценки моделей. 

Ниже будет приведён довольно сухой код с огромным количеством комментариев. На мой взгляд, для человека, который как следует изучил туториал, большая часть этого кода (исключим куски, где я явно говнокодю) должна быть вроде как понятна (но это неточно). 

## 1. Подготовка

По классике добавляем токен для доступа к своей страничке и id приложения. Если вы уже не понимаете, что происходит, марш читать вводную статью! 

In [1]:
version = 'v30-12-17'  # Версия данных для сохранения на диск 
token= "64892a92740305e884543c02c538b1d2c2941a323d86e8c481fc64e5333e406b34e53a0a5dc879d005152"
myid = "6045249"

Теперь подгрузим несколько пакетов, которые пригодятся нам для дальнейшей работы.

In [2]:
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нанего
                       # сборщика данных из-за слишком частых запросов
    
# Пакет для красивых циклов. При желании его можно отключить и удалить из всех циклов 
# команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

import pickle   # Пакет для сохранения и подгрузки данных 
import re       # Пакет для работы с регулярными выражениями 

import datetime            # Пакет для работы с датами и временем
from datetime  import date # вытащим из пакета только эту функцию (это будет удобно при работе со стенкой и фото)

Напишем функцию для наших запросов к вк, которая будет пригодна для всех методов и параметров. Это хорошая функция. В принципе, все функции должны быть коротенькими. К сожалению, ближе к концу парсинга начнется функциональная жесть в виде киломметровых функций. Я хотел бы принести за них свои извинения. 

In [3]:
def vk_download(method,parameters,token = token):
    url = 'https://api.vk.com/method/'+method+'?'+parameters+'&access_token='+str(token)
    response = requests.get(url) 
    return(response.json())

# Проверяем всё ли работает на своём имени.
vk_download('users.get','user_ids='+myid)

{'response': [{'first_name': 'Филипп',
   'last_name': 'Ульянкин',
   'uid': 6045249}]}

Подгружаем файлы с id-шниками интересующих нас людей из файликов с разметкой. Это наш первый источник информации. 

In [4]:
# файлы, в которых лежат id и другие размеченые данные называются Total_20XX 
# Заведем вектор со всеми годами, которые есть у нас в распоряжении
file_names = ['2012','2013','2014','2015','2016','2017']
uids = [ ] # Вектор, куда мы будем записывать все уиды  (user id)

# Проходимся по всем файлам и забираем uid
for file in file_names: 
    cur_df = pd.read_excel('../Total_razmetka/Total_' + file + '.xlsx') # В этой папке файлы...
    cur_ids = [ ]
    # В табличке в графе uids лежат нащи уиды 
    for item in cur_df['uids']:
        try:
            cur_ids.append(int(item)) # Пробуем добавить в вектор id
        except:
            pass  # Если возникла ошибка, у нас NA. Не делаем ничего. 
        
    uids.extend(cur_ids)
    
print(len(uids))      # Такс такс такс.... Сколько у нас уидов 
print(len(set(uids))) # Проверка нет ли одинаковых uid в векторе 
                      # Команда set() превращает вектор в множество и удаляет дубликаты
    
# Какие-то ребята затесались в таблички по два раза! Например, Антон Гисин... 

438
433


Пройдёмся по всем экономовским группам и соберём себе id всех их участников. **Интересующие нас группы:**
* иканам  https://vk.com/ikanam
* 2017  https://vk.com/economist2017
* 2016  https://vk.com/ikanamchik
* 2015  https://vk.com/efka2015
* 2014  https://vk.com/ekfa14
* 2013  https://vk.com/club102346076
* 2012  https://vk.com/economy.rane
* 2011  https://vk.com/club29313387
* 2010  https://vk.com/club19743751

In [5]:
# Достаём участиков каждой из групп по её айдищнику
groups_id = ['ikanam','economist2017','ikanamchik','efka2015','ekfa14','102346076','economy.rane',
             '29313387','19743751']

users = [vk_download('groups.getMembers','group_id='+ids)['response']['users'] for ids in groups_id]

# Объединим все id, добытые из групп в один список
uids_1 = [ ]
for item in users:
    uids_1.extend(item)
print('В списке оказалось ',len(uids_1),'юзеров')

# Оставим только уникальные id 
uids_set = set(uids_1)
print('В списке оказалось',len(uids_set),'уникальных юзеров')

# Объединим все собранные id в один вектор 
all_uids = uids_set | set(uids) # Объединяем множества id 
all_uids = list(all_uids)       # Превращаем множество в вектор 
print('Юзеров из табличек загружено: ', len(set(uids)))
print('Всего юзеров в выборке: ', len(all_uids))

В списке оказалось  1109 юзеров
В списке оказалось 613 уникальных юзеров
Юзеров из табличек загружено:  433
Всего юзеров в выборке:  706


Подтянем для каждого уида информацию про имена, пол и ссылку на фотку. Впоследствии эта основная информация часто будет нам нужна. Сохраним её в виде словарика с ключами в виде айдишников. Когда мы будем строить визуализации, по этим самым ключам нам будет удобно вызывать нужную информацию о иканамовцах. 

In [6]:
# Проходимся по каждому id из списка и добавляем в словарики основную информацию 
# по каждому иканамовцу 

ikanam_info = { }   # В этом словарике будем хранить информацию про иканамовцев

for ikanamovec in tqdm_notebook(all_uids): 
    time.sleep(0.4)   # Не забываем отдохнуть между запросами, чтобы вк не злился! 
    try:
        cur_info = { } # Информация по текущему другу будет храниться тут 
        res = vk_download('users.get','user_ids='+str(ikanamovec)+'&fields=sex,photo')['response'][0]
        cur_info['name'] = res['last_name'] + ' ' + res['first_name']
        cur_info['sex'] = res['sex']
        cur_info['photo'] = res['photo']
        
        # закидываем новую информацию в словарик 
        ikanam_info[ikanamovec] = cur_info   
    except:
        # Если скачка не удалась, выведем сообщение об ошибке
        # (По идее, мы хорошо обработали до этого всех юзеров и ошибок не будет)
        print('Ощибка с ', ikanamovec)

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))

In [7]:
# Страшная команда для сохранения файлика :) 
with open('ikanam_info_' + version, 'wb') as f:
    pickle.dump(ikanam_info, f)

## 2. Качаем данные по каждой странице! 

Обычно данные собирают и хранят в максимально сыром виде. Обработка сырых данных бывает очень сложной процедурой. При очистке можно случайно повредить что-то важное. Именно поэтому мы будем сохранять все интересующие нас данные в грязном необработанном формате и в чистом, обработанном. Если вам не понравятся фичи, которые я сварю, вы сможете взять грязные данные и сварить на их основе что-нибудь своё. 

### 2.1 Скачиваем данные по профилям 

Внимательно поизучаем то какие именно штуки можно выгрузить по каждой страничке [в документации](https://vk.com/dev/objects/user) и поделим всё это чуда на пять разных категорий. После будем скачивать данные по каждой категории и работать с ними. 

Прошу любить и жаловать! Наши категории: 

In [8]:
# Основные параметры
first_par = ['bdate','city','home_town','relation', 'sex']

# Есть ли возможность получить данные (например, открыты ли комменты или аудио)
availible_functions = ['can_post','can_see_all_posts','can_see_audio','can_write_private_message','has_mobile', \
                       'has_photo','wall_comments']
# Вторичные параметры
second_par = ['about','activities','books','connections','contacts','country','education','exports',\
                    'followers_count', 'games', 'interests', 'movies','music','quotes', 'site', 'tv']

# Важные списки
first_spisok = ['counters','personal','last_seen']
# Вторичные списки
second_spisok = ['schools', 'universities', 'occupation', 'relatives']

Для скачки данных нам надо будет подставлять в функцию для скачки в параметры что именно мы хотим получить. Напишем небольшую функцию, которая н основе векторов выше будет формировать запросы для нашей функции в соотвествии с документацией. 

In [9]:
def parametr_creator(list):
    param = ''
    for obj in list:
        param += obj + ','
    return param

parametr_creator(availible_functions)

'can_post,can_see_all_posts,can_see_audio,can_write_private_message,has_mobile,has_photo,wall_comments,'

Скачаем данные по каждый из пяти представленных категорий в отдельный файл и сохраним их.

In [10]:
%%time
# Внимание! Выбран не очень удобный способ скачки. Можно проще. Не сломайте себе мозг,
# если решите изучить этот код во всех подробностях...

# Создаём свой инфа вектор для каждого рода информации
profile_first = [ ]
profile_second = [ ]
profile_availible = [ ]
profile_first_spisok = [ ]
profile_second_spisok = [ ]

spiski = [first_par, second_par, availible_functions, first_spisok, second_spisok]
infa = [profile_first, profile_second, profile_availible, profile_first_spisok, profile_second_spisok]

# Первый цикл идёт по всем юзерам из списка. Второй цикл идёт по парам (название параметра, вектор для записи)
# В итоге на выходе получаем заполненные вектора из вектора spisok. 

# На случай, если красивая строка для цикла вдруг не работает 
# Надо установить виджет отсюда:  https://ipywidgets.readthedocs.io/en/stable/user_install.html

for user in tqdm_notebook(all_uids):
    for a, b in zip(spiski,infa):
        time.sleep(0.4) # Временная заглушка
        we_want = parametr_creator(a) # Превращаем вектор в текст
        current = vk_download('users.get','user_ids='+str(user)+'&fields='+we_want)
        b.append(current)
        
# Установим специальный пакет и будем отправлять себе в телеграм сообщения, когда код закончит работу 
# Подробнее смотри: https://habrahabr.ru/post/339682/
# Да! Это очень пафосно, писать себе из питона в телеграм :3
!telegram-send "Скрипт по скачке данных. Создан infa-вектор"

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))


CPU times: user 1min 7s, sys: 4.3 s, total: 1min 11s
Wall time: 36min 57s


Грязные данные по эконому оказались в наших руках. Все 5 разных словарей с разной информацией. Сохраним грязные данные на наш компьютер, а после приступим к их обработке. 

In [11]:
# сохраняем грязные данные 
with open('vk_profile_data_' + version, 'wb') as f:
    pickle.dump(infa, f)

### 2.2 Варим фичи по профилям 

Теперь нужно переработать всё, что мы скачали из вк в один большой красивый датафрэйм (красивую табличку). Начинаем вручную всё перелопачивать... 

Обратите внимание на то, что мы не хотим получить на выходе уже готовую таблицу с фичами для обучения. Мы хотим максимально очистить её от мусора. В каких-то определённых местах это действительно приведёт к варке годных фичей, а где-то нет. 

> Сразу же договоримся об обозначениях. Будем называть фичи следущим образом:  `Откуда_имя_тип`

Например, название `profile_facebook_dummy` будет означать, что наша фича взята из профиля человека, принимает значения 0 или 1 и описывает есть ли у человека в профиле ссылка на facebook. Такая нотация позволит нам не сойти с ума раньше времени. 

Типы фичей: 

* dummy (принимают значения 0 или 1)
* categ (категориальная фича, принимает значения из некоторого конечного множества) 
* cnt (фича - счётчик, принимает только целочисленные значения)
* float (действительное число) 
* Фича без подобного суффикса — что-то особое

In [12]:
# Создаём таблицу, в которую мы запишем все созданные фичи
df_profile = pd.DataFrame() 

#### 2.2.1. Первая инфа

В первой части нашей инфы лежат основные данные о человеке. Город, имя, пол и т.п. Взглянем на какой-нибудь кусочек данных. 

In [13]:
infa[0][0]['response'] # Красота! 

[{'bdate': '27.9.1997',
  'city': 1,
  'first_name': 'Ксения',
  'home_town': 'Кострома',
  'last_name': 'Кулак',
  'relation': 0,
  'sex': 1,
  'uid': 22261760}]

In [14]:
# Все ответы на наши запросы лежат внутри словариков в графе респонс
# Для каких-то ситуаций (например, пользователь удалил страницу) 
# респонса может не быть. Посмотрим на такие случаи более пристально.

for item in infa[0]:
    try:
        item['response']  # Проверяем всё ли корректно скачалось 
    except:
        print(item)       # Если где-то нет респонса, посмотрим как выглядит этот item 

In [15]:
# Очистим данные от респонсов.
inf_1 = [item['response'][0] for item in infa[0]]
# Поприветствуем первый датафрейм!
df_1 = pd.DataFrame(inf_1)

print('Размер таблицы: ', df_1.shape)
df_1.head()

Размер таблицы:  (706, 10)


,bdate,city,deactivated,first_name,home_town,last_name,relation,relation_partner,sex,uid
0,27.9.1997,1.0,NaN,Ксения,Кострома,Кулак,0.0,NaN,1,22261760
1,18.4.1993,1.0,NaN,Валера,NaN,Левин,NaN,NaN,2,295389188
2,13.2.1917,1516109.0,NaN,Юрий,"Залупкино, Московская область",Сарафанюк,0.0,NaN,2,174168070
3,NaN,1.0,NaN,Светлана,Белгород,Скибенко,0.0,NaN,1,137312267
4,NaN,0.0,NaN,Антон,NaN,Антонов,NaN,NaN,2,47220748


In [16]:
# Посмотрим на пропуски в данных
df_1.isnull().sum()

bdate               178
city                 33
deactivated         673
first_name            0
home_town           332
last_name             0
relation            332
relation_partner    682
sex                   0
uid                   0
dtype: int64

Что мы видим и что бы нам хотелось видеть?

Во-первых, видим, что пацана из залупкино сразу можно отчислять за глупые шутки... 

1. bdate — Дата рождения. Она приведена в довольно странном формате. Многие люди вообще не удосужились её указать. Некоторые люди указали фиктивные даты рождения. Разобьём колонку на дни, месяцы и годы рождения, а также сразу построим фичу, которая отвечает за правдивость даты. Если указан год меньше 1990, будем считать, что человек лжёт. 
2. deactivated — Переменная, которая говорит удалил ли человек свою страницу. Если да, то она принимает значение 1. Заменим все NaN на нули. Страница же не удалена...
3. city — id города, который указан у человека в профиле. В документации по методам API чёрным по белому написано, что для того, чтобы восстановить название города надо использовать метод `database.getCitiesByld`. Восстановим настоящие названия городов. 
4. sex — Пол. Переведём его в шкалу 0-1, где 1 — мужчина
    * 1 — женский; 
    * 2 — мужской. 
5. relation - Семейное положение пользователя. Заменим все цифры на названия. Что какая цифра означает мы быстро забудем. 
    * 1 — не женат/не замужем;
    * 2 — есть друг/есть подруга;
    * 3 — помолвлен/помолвлена;
    * 4 — женат/замужем;
    * 5 — всё сложно;
    * 6 — в активном поиске;
    * 7 — влюблён/влюблена;
    * 8 — в гражданском браке;
    * 0 — не указано.     
6. Оставим оставшиеся фичи нетронутыми
    * relation_partner — с кем у человека семейное положение
    * first_name — Имя.
    * last_name — Фамилия.
    * uid — id человека. Позже мы будем по uid объединять таблицы. 
    * home_town — родной город.

In [17]:
# 0. Не трогаем 

df_profile['uid'] = df_1['uid']
df_profile['profile_first_name'] = df_1['first_name']
df_profile['profile_last_name'] = df_1['last_name']


# 1. Разобьём колонку Дата рождения на колонку год, месяц, день рождения. 
# Сделаем это убого и прямо влоб, циклом 

year, month, day, fyear = [ ], [ ], [ ], [ ] # Сюда записываем результат
for item in df_1.bdate:
    try:
        potential = item.split('.')      # Пробуем разбить элемент на части по .
        if len(potential[-1]) == 4:      # Если длина последней части 4, это год 
            ye = int(potential[-1])
            year.append(ye)              # Запоминаем год,
            month.append(potential[-2])  # месяц 
            day.append(potential[-3])    # и день
            fyear.append(int(ye < 1990)) # Правдивый ли год указан  
        else: 
            year.append(None)            # Ужели длина не 4, год не указан 
            month.append(potential[-1])  # Запоминаем месяц и 
            day.append(potential[-2])    # день    
            fyear.append(None)
    except:
        year.append(None)                # Не смогли разбить на части? 
        month.append(None)               # Значит такого элемента нет,
        day.append(None)                 # Везде записываем пропуски.
        fyear.append(None)

df_profile['profile_byear'] = year             # Добавляем колонку отвечающую за год 
df_profile['profile_bmonth'] = month           # месяц 
df_profile['profile_bday'] = day               # и день
df_profile['profile_false_year_dummy'] = fyear       # Правдивый ли год указал пользователь

# 2. Заменим в колонке deactivated все пропуски на нули
df_profile['deactivated_dummy'] = df_1.deactivated
df_profile['deactivated_dummy'].fillna(0,inplace=True)

# 3. Восстанавливаем названия городов! 
# Понимаем какая цифра относится к какому городу, чтобы заменить цифры на города в датафрейме
goroda  = { }
for item in set(df_1.city):
    time.sleep(0.4)
    if not (pd.isnull(item))|(item==0):
        goroda.update({item : vk_download('database.getCitiesById','city_ids='+str(item))\
                       ['response'][0]['name']})

# Заменяем цифры на города. Искусственно добавляем в словарь для замены нулевой элемент.
goroda.update({0:'Город не указан'})
df_1.city.replace(goroda,inplace = True)
df_profile['profile_city'] = df_1['city']
# Тут же запишем указанный родной город 
df_profile['profile_home_town'] = df_1['home_town']


# 4. Создаэм дамми-переменную 'Мужчина'
df_profile['profile_male_dummy'] = df_1['sex'] - 1 

# 5. Создаём колонку с семейным положением. 
df_profile['profile_relation_cat'] = df_1['relation'].replace({
    1 : 'не женат/не замужем',
    2 : 'есть друг/есть подруга',
    3 : 'помолвлен/помолвлена',
    4 : 'женат/замужем',
    5 : 'всё сложно',
    6 : 'в активном поиске',
    7 : 'влюблён/влюблена',
    8 : 'в гражданском браке',
    0 : 'не указано' })

df_profile['profile_relation_partner'] = df_1['relation_partner']

df_profile.head()

,uid,profile_first_name,profile_last_name,profile_byear,profile_bmonth,profile_bday,profile_false_year_dummy,deactivated_dummy,profile_city,profile_home_town,profile_male_dummy,profile_relation_cat,profile_relation_partner
0,22261760,Ксения,Кулак,1997.0,9,27,0.0,0,Москва,Кострома,0,не указано,NaN
1,295389188,Валера,Левин,1993.0,4,18,0.0,0,Москва,NaN,1,NaN,NaN
2,174168070,Юрий,Сарафанюк,1917.0,2,13,1.0,0,Montego Bay,"Залупкино, Московская область",1,не указано,NaN
3,137312267,Светлана,Скибенко,NaN,None,None,NaN,0,Москва,Белгород,0,не указано,NaN
4,47220748,Антон,Антонов,NaN,None,None,NaN,0,Город не указан,NaN,1,NaN,NaN


#### 2.2.2 Вторая инфа 

Во второй части инфы лежит всякая второсортная информация. Поглядим на неё чуть более подробно. 

In [18]:
infa[1][0]['response']

[{'about': '',
  'activities': '',
  'books': '',
  'country': 1,
  'faculty': 6692,
  'faculty_name': 'Экономический факультет',
  'first_name': 'Ксения',
  'followers_count': 133,
  'games': '',
  'graduation': 0,
  'home_phone': '',
  'interests': '',
  'last_name': 'Кулак',
  'mobile_phone': '',
  'movies': '',
  'music': '',
  'quotes': '',
  'site': 'www.vkontakte.ru/id22261760',
  'tv': '',
  'uid': 22261760,
  'university': 92,
  'university_name': 'РАНХиГС при Президенте РФ (АНХ при Правительстве РФ, РАГС при Президенте РФ)'}]

In [19]:
inf_2 = [item['response'][0] for item in infa[1]] # Очистим данные от респонсов.
df_2 = pd.DataFrame(inf_2)                        # Создаём черновую табличку 

print('Размер таблицы: ', df_2.shape)             # Такс такс такс, какие размеры у неё
df_2.head() 

Размер таблицы:  (706, 32)


,about,activities,books,country,deactivated,education_form,education_status,exports,facebook,facebook_name,...,movies,music,quotes,site,skype,tv,twitter,uid,university,university_name
0,,,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,www.vkontakte.ru/id22261760,NaN,,NaN,22261760,92.0,РАНХиГС при Президенте РФ (АНХ при Правительст...
1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,295389188,NaN,NaN
2,,Читаю книжки сижу на мужских шишках,,228.0,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,NaN,,NaN,174168070,0.0,
3,,,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,https://www.instagram.com/_____sa___/,s19sveta98a,,NaN,137312267,92.0,РАНХиГС при Президенте РФ (АНХ при Правительст...
4,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,47220748,NaN,NaN


In [20]:
df_2.isnull().sum()                               # А где пропуски? 

about               332
activities          332
books               332
country              33
deactivated         673
education_form      556
education_status    554
exports             705
facebook            686
facebook_name       686
faculty             332
faculty_name        332
first_name            0
followers_count      33
games               332
graduation          332
home_phone          231
instagram           571
interests           332
last_name             0
livejournal         705
mobile_phone        268
movies              332
music               332
quotes              332
site                 33
skype               603
tv                  332
twitter             670
uid                   0
university          332
university_name     332
dtype: int64

Сразу же видим, что пацан из Залупкино продолжает отжигать. Бьюсь об заклад, что он подписан на МДК. 

В таблице по многим переменным зашкаливающее количество пропусков. Обдумаем более подробно какие переменные стоит оставить, а какие лучше удалить. 

* about, activities, books, interests, movies, music, quotes, tv, games  — Всё это текстовые переменные  Текстовые переменные. Вынесем все текстовые переменные в отдельную таблицу и сохраним её в папочку с текстовой информацией. Обработке такой информации будет посвящён отдельный код. 

In [21]:
df_text = df_2[['about', 'activities', 'books', 'interests', 'movies', 'music', 'quotes', 'tv','games','uid']]
# Переименуем колонки в этой таблице так, чтобы потом было удобно доставать переменные и решать что с ними в итоге
# делать 
df_text.columns = ['about_about', 'about_activities', 'about_books', 
                   'about_interests', 'about_movies', 'about_music', 
                   'about_quotes', 'about_tv','about_games','uid']

df_2.drop(['about', 'activities', 'books', 'interests', 'movies',
                   'music', 'quotes', 'tv','games'], axis=1, inplace = True)

df_text.to_csv('profile_about.csv')

* education_form, education_status, exports, livejournal — эти переменные пропущены практически у всех людей в выборке, кроме одного какого-то психопата. Выясним, кто этот чудик, а потом удалим эти переменные.

> Как выяснилось, этот человек — я. Более интересно, у кого в 2017 году есть Живой Журнал. Но на этот вопрос я не дам вам ответа. Должен же у вас появиться исследовательский запал для скачивания этой тетрадки и своего собственного исследования...

* Колонки university и university_name дублируют друг-друга В первой указывается цифра, соответствующая вузу. Удалим первую. Аналогично поступим с графами faculty и faculty_name. Также выбросим графу graduation. Она будет дублировать колонку "год поступления", которая появится у нас немного позже. 

In [22]:
# Удаляем эти переменные!
df_2.drop(['education_form', 'education_status', 'exports', 'livejournal'], axis=1, inplace = True)

# Удаляем колонки - дубликаты.
df_2.drop(['university','faculty','facebook_name','graduation'], axis=1, inplace = True)

Смотрим что у нас осталось после такого безжалостного отсева мусора. 

1. country - по аналогии с городом превратим все цифры в нормальные названия.
2. Все колонки с другими соцсетями такими как ссылки на instagram и мобильными телефонами преобразуем в дамми-переменные, которые характеризуют наличие такой ссылки в профиле. При более большой выборке, имело бы смысл выявить хохмачей вроде парня из Залупкино, но мы не будем делат это, так как данных мало, а фичей будет много. Часть фичей придётся выбросить на помойку. 
3. Колонки с количеством фоловеров, факультетом и вузом оставим пока что нетронутыми.  

In [23]:
# 1. Разбираемся со странами также как с городами 
strana  = { }
for item in set(df_2.country):
    time.sleep(0.4)
    if not (pd.isnull(item))|(item==0):
        strana.update({item : vk_download('database.getCountriesById','country_ids='+str(item))\
                          ['response'][0]['name']})

# Заменяем цифры на страны. Искусственно добавляем в словарь для замены нулевой элемент.
strana.update({0:'Страна не указана'})
df_2.country.replace(strana,inplace = True)
df_profile['profile_country'] = df_2.country


# 2. Преобразуем все колонки с отсылками в другие сети в то, что в них что-то указано

def social_replace(df, colname):
    # Везде, где нет None или незаполненной ячейки, ставлю 1
    df.ix[(df[colname].isnull() == False)&(df[colname] != ''), colname] = 1 
    # Все None и пустоты меняем на нули 
    df.ix[(df[colname].isnull() == True)|(df[colname] == ''), colname] = 0
    return df[colname]

df_profile['pofile_facebook_dummy'] = social_replace(df_2,'facebook')
df_profile['profile_instagram_dummy'] =social_replace(df_2,'instagram')
df_profile['profile_skype_dummy'] = social_replace(df_2,'skype')
df_profile['profile_twitter_dummy'] = social_replace(df_2,'twitter')
df_profile['profile_home_phone_dummy'] = social_replace(df_2,'home_phone')
df_profile['profile_mobile_phone_dummy'] = social_replace(df_2,'mobile_phone')
df_profile['profile_site_dummy'] = social_replace(df_2,'site')


# 3. Перекидывем хорошие колонки в нашу итоговую табличку  
df_profile['profile_folowers_cnt'] = df_2['followers_count']   
df_profile['profile_university_str'] = df_2['university_name']
df_profile['profile_faculty_str'] = df_2['faculty_name']

df_profile.head()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,uid,profile_first_name,profile_last_name,profile_byear,profile_bmonth,profile_bday,profile_false_year_dummy,deactivated_dummy,profile_city,profile_home_town,...,pofile_facebook_dummy,profile_instagram_dummy,profile_skype_dummy,profile_twitter_dummy,profile_home_phone_dummy,profile_mobile_phone_dummy,profile_site_dummy,profile_folowers_cnt,profile_university_str,profile_faculty_str
0,22261760,Ксения,Кулак,1997.0,9,27,0.0,0,Москва,Кострома,...,0,0,0,0,0,0,1,133.0,РАНХиГС при Президенте РФ (АНХ при Правительст...,Экономический факультет
1,295389188,Валера,Левин,1993.0,4,18,0.0,0,Москва,NaN,...,0,0,0,0,0,0,0,53.0,NaN,NaN
2,174168070,Юрий,Сарафанюк,1917.0,2,13,1.0,0,Montego Bay,"Залупкино, Московская область",...,0,0,0,0,0,1,0,36.0,,
3,137312267,Светлана,Скибенко,NaN,None,None,NaN,0,Москва,Белгород,...,0,0,1,0,0,0,1,117.0,РАНХиГС при Президенте РФ (АНХ при Правительст...,
4,47220748,Антон,Антонов,NaN,None,None,NaN,0,Город не указан,NaN,...,0,0,0,0,0,0,0,47.0,NaN,NaN


#### 2.2.3 Третья инфа

В третьей части инфы лежит несколько важных переменных. Можно ли смотреть фото человека, открыты ли у него аудио, личные сообщения, есть ли у него аватар и т.п.

In [24]:
infa[2][0]['response']

[{'can_post': 1,
  'can_see_all_posts': 1,
  'can_see_audio': 1,
  'can_write_private_message': 1,
  'first_name': 'Ксения',
  'has_mobile': 1,
  'has_photo': 1,
  'last_name': 'Кулак',
  'uid': 22261760,
  'wall_comments': 1}]

In [25]:
# Очистим данные от респонсов.
inf_3 = [item['response'][0] for item in infa[2]]
df_3 = pd.DataFrame(inf_3)

# сразу же удалим две лишние колонки
df_3.drop(['first_name','last_name','deactivated','uid'],axis=1,inplace=True)

print('Размер таблицы: ', df_3.shape)
df_3.head()

Размер таблицы:  (706, 7)


,can_post,can_see_all_posts,can_see_audio,can_write_private_message,has_mobile,has_photo,wall_comments
0,1,1.0,1.0,1,1.0,1,1.0
1,0,0.0,0.0,1,1.0,1,1.0
2,1,1.0,1.0,0,1.0,1,1.0
3,1,1.0,1.0,1,1.0,1,1.0
4,0,0.0,0.0,1,1.0,0,1.0


In [26]:
df_3.isnull().sum()

can_post                      0
can_see_all_posts            33
can_see_audio                33
can_write_private_message     0
has_mobile                   33
has_photo                     0
wall_comments                33
dtype: int64

Неимоверное везение! Все переменные — дамми. Часть переменных субъективная. 

In [27]:
# Открыта ли для моих записей стена пользователя 
df_profile['meprofile_can_post_dummy'] = df_3.can_post
# Открыты ли для просмотра мной стена пользователя
df_profile['meprofile_can_see_all_posts_dummy'] = df_3.can_see_all_posts
# Открыты ли для меня аудио пользователя
df_profile['meprofile_can_see_audio_dummy'] = df_3.can_see_audio
# Могу ли я писать пользователю сообщения
df_profile['meprofile_can_write_private_message_dummy'] = df_3.can_write_private_message
# Известен ли вк мобильник пользователя
df_profile['profile_has_mobile_dummy'] = df_3.has_mobile
# Есть ли у пользоваетеля ава 
df_profile['profile_has_ava_dummy'] = df_3.has_photo
# Включены ли на стене комментарии 
df_profile['profile_wall_comments_dummy'] = df_3.wall_comments

#### 2.2.4 Четвёртая инфа

В четвёртой части инфы лежит несколько важных списков. В основном это переменные-счётчики (сколько фоток на странице, видео на странице и т.п.) и разные категориальные переменные. Обработка этой части профиля самая хардовая.  

In [28]:
infa[3][0]['response']

[{'counters': {'albums': 1,
   'audios': 1612,
   'followers': 133,
   'friends': 155,
   'gifts': 71,
   'mutual_friends': 35,
   'notes': 1,
   'online_friends': 40,
   'pages': 100,
   'photos': 86,
   'subscriptions': 1,
   'user_photos': 96,
   'videos': 308},
  'first_name': 'Ксения',
  'last_name': 'Кулак',
  'last_seen': {'platform': 4, 'time': 1514645045},
  'personal': {'langs': ['Русский', 'English']},
  'uid': 22261760}]

In [29]:
# Очистим данные от респонсов.
inf_4 = [item['response'][0] for item in infa[3]]

Начнём потихоньку заполнять наш новый словарик, на основе которого будет создан четвёртый датафрейм.

1. counters — вытаскиваем переменные-счётчикию. Для них пишем отдельную супермега-функцию доставатель. 
2. last_platform — с какого устройства в последний раз заходил юзер. Ответ выдаётся в виде цифр. Раскодируем их в соотвествии с документацией после того как построим таблицу. 
3. last_seen — как давно в последний раз заходил пользователь 
    * 1 — мобильная версия;
    * 2 — приложение для iPhone;
    * 3 — приложение для iPad;
    * 4 — приложение для Android;
    * 5 — приложение для Windows Phone;
    * 6 — приложение для Windows 10;
    * 7 — полная версия сайта.
4. personal — персональные данные пользователя. От политических предпочтений до языков. Очень частно не указаны. Тоже выдаются в виде цифр, которые придется расшифровывать. 
    * political (integer) — политические предпочтения. Возможные значения:
        * 1 — коммунистические;
        * 2 — социалистические;
        * 3 — умеренные;
        * 4 — либеральные;
        * 5 — консервативные;
        * 6 — монархические;
        * 7 — ультраконсервативные;
        * 8 — индифферентные;
        * 9 — либертарианские.
    * langs (array) — языки.
    * religion (string) — мировоззрение.
    * inspired_by (string) — источники вдохновения.
    * people_main (integer) — главное в людях. Возможные значения:
        * 1 — ум и креативность;
        * 2 — доброта и честность;
        * 3 — красота и здоровье;
        * 4 — власть и богатство;
        * 5 — смелость и упорство;
        * 6 — юмор и жизнелюбие.
    * life_main (integer) — главное в жизни. Возможные значения:
        * 1 — семья и дети;
        * 2 — карьера и деньги;
        * 3 — развлечения и отдых;
        * 4 — наука и исследования;
        * 5 — совершенствование мира;
        * 6 — саморазвитие;
        * 7 — красота и искусство;
        * 8 — слава и влияние;
    * smoking (integer) — отношение к курению. Возможные значения:
        * 1 — резко негативное;
        * 2 — негативное;
        * 3 — компромиссное;
        * 4 — нейтральное;
        * 5 — положительное.
    * alcohol (integer) — отношение к алкоголю. Возможные значения:
        * 1 — резко негативное;
        * 2 — негативное;
        * 3 — компромиссное;
        * 4 — нейтральное;
        * 5 — положительное.

In [30]:
# Функция, которая ищет список для его распаковки. 
# Если она не находит список, распаковывается пропуск. 
def tryer(obj_in, dct_out, for_try, is_dct = False, bad_case = np.nan):
    try:
        it = obj_in[for_try]
        if is_dct:
            dct_out.update(it)
        else:
            dct_out.update({for_try : it})
    except Exception:
        if is_dct:
            dct_out.update(bad_case)
        else:
            dct_out.update({for_try : bad_case})
    return dct_out

# Если для человека счётчик не найден, будем записывать в ячейку None 
bad_counters_case = {'albums':  np.nan, 'audios': np.nan, 'followers': np.nan, 'friends': np.nan,
                     'gifts': np.nan, 'groups': np.nan, 'mutual_friends': np.nan, 'notes': np.nan,
                     'online_friends': np.nan, 'pages': np.nan, 'photos': np.nan, 'subscriptions': np.nan,
                     'user_photos': np.nan, 'videos': np.nan}

dct_inf_4 = [ ]  # Список для наших словариков по каждому человеку 

# Проходимся циклом по всем людям и распаковываем информацию по полочкам 
for peace_of_inf in inf_4:
    dct_4 = { } # Сюда будем записывать всю инфу по текущему юзеру 
    dct_4.update({'uid':peace_of_inf['uid']}) # Записали uid, чтобы после объединить таблицы
    
    # 1. Прогоняем нашу страшную функцию для распаковки счётчиков
    # На выход получаем заполненный счётчиками dct_4 
    tryer(peace_of_inf, dct_4, 'counters', is_dct = True, bad_case=bad_counters_case)
    
    # 2. 3. Точно также распаковываем устройства 
    try:
        dct_4.update({'last_platform' : peace_of_inf['last_seen']['platform']})
        dct_4.update({'last_time' : peace_of_inf['last_seen']['time']})
    except: # ошибка обычно происходит, если страница удалена  
        dct_4.update({'last_platform' : np.nan})
        dct_4.update({'last_time' : np.nan})
     
    # 4. Пробуем достать персональные данные о человеке, если они вообще есть.
    try:
        it = peace_of_inf['personal']
    except Exception:
        it = { }

    # Проходимся по всему меню из данных и для каждого типа запиливаем свою колонку 
    a = { }
    names = ['political','langs','religion','inspired_by','people_main',
                    'life_main','smoking','alcohol']

    for name in names:
        tryer(it, a,  name)
    # Закидываем всё найденное в итоговый словарь
    dct_4.update(a)
    # Добавляем словарь в вектор из словарей
    dct_inf_4.append(dct_4)
    
    
# Строим итоговую таблицу, смотрим на число пропусков и её размеры
df_4 = pd.DataFrame(dct_inf_4)

print('Размер таблицы: ', df_4.shape)
df_4.head()

Размер таблицы:  (706, 25)


,albums,alcohol,audios,followers,friends,gifts,groups,inspired_by,langs,last_platform,...,pages,people_main,photos,political,religion,smoking,subscriptions,uid,user_photos,videos
0,1.0,NaN,1612.0,133.0,155.0,71.0,NaN,NaN,"[Русский, English]",4.0,...,100.0,NaN,86.0,NaN,NaN,NaN,1.0,22261760,96.0,308.0
1,0.0,NaN,0.0,53.0,198.0,NaN,NaN,NaN,NaN,2.0,...,54.0,NaN,42.0,NaN,NaN,NaN,5.0,295389188,NaN,27.0
2,1.0,5.0,463.0,36.0,95.0,30.0,34.0,NaN,"[Русский, Shqip]",4.0,...,22.0,4.0,5.0,1.0,Буддизм,5.0,0.0,174168070,0.0,79.0
3,12.0,NaN,928.0,117.0,337.0,61.0,186.0,NaN,NaN,1.0,...,139.0,NaN,208.0,NaN,NaN,NaN,1.0,137312267,3.0,118.0
4,0.0,NaN,0.0,47.0,113.0,NaN,NaN,NaN,NaN,4.0,...,41.0,NaN,8.0,NaN,NaN,NaN,0.0,47220748,NaN,113.0


Отлично! Осталось немного подчистить табличку и дело с концом!

In [31]:
df_4_clean = pd.DataFrame() 

# По колонке uid мы будем объядинять таблички. 
df_4_clean['uid'] = df_4.uid
# Переименовываем все счётчики
df_4_clean['profile_albums_cnt'] = df_4.albums
df_4_clean['profile_audio_cnt'] = df_4.audios
df_4_clean['profile_followers_cnt'] = df_4.followers
df_4_clean['profile_friends_cnt'] = df_4.friends
df_4_clean['profile_gifts_cnt'] = df_4.gifts
df_4_clean['profile_groups_cnt'] = df_4.groups
df_4_clean['profile_mutual_friends_cnt'] = df_4.mutual_friends
df_4_clean['profile_notes_cnt'] = df_4.notes
df_4_clean['profile_photos_cnt'] = df_4.photos
df_4_clean['profile_subscriptions_cnt'] = df_4.subscriptions
df_4_clean['profile_user_photos_cnt'] = df_4.user_photos
df_4_clean['profile_videos_cnt'] = df_4.videos
df_4_clean['profile_pages_cnt'] = df_4.pages

# Расшифровываем все категориальный фичи, связанные с политикой и алкоголем
# Алкогольная и сигаретная фичи, это оценки по шкале от 1 до 5. 
# Они разумно упорядочены, не будем их менять
df_4_clean['profile_alco_love_cat'] = df_4.alcohol
df_4_clean['profile_smoke_love_cat'] = df_4.smoking

# Колонки религия и источники вдохновения заполняются людьми самостоятельно
# и сильно разряжены. Оставим пока что их в исходном виде 
df_4_clean['profile_religion_str'] = df_4.religion
df_4_clean['profile_inspired_by_str'] = df_4.inspired_by

# Главное в жизни, политические взгляды и главное в людях раскодируем в текст
df_4_clean['profile_life_main_cat'] = df_4.life_main.replace({
    1 : 'семья и дети',
    2 : 'карьера и деньги',
    3 : 'развлечения и отдых',
    4 : 'наука и исследования',
    5 : 'совершенствование мира',
    6 : 'саморазвитие',
    7 : 'красота и искусство',
    8 : 'слава и влияние'
    })

df_4_clean['profile_people_main_cat'] = df_4.people_main.replace({
    1 : 'ум и креативность',
    2 : 'доброта и честность',
    3 : 'красота и здоровье',
    4 : 'власть и богатство',
    5 : 'смелость и упорство',
    6 : 'юмор и жизнелюбие' 
    })

df_4_clean['profile_political_cat'] = df_4.political.replace({
    1 : 'коммунистические',
    2 : 'социалистические',
    3 : 'умеренные',
    4 : 'либеральные',
    5 : 'консервативные',
    6 : 'монархические',
    7 : 'ультраконсервативные',
    8 : 'индифферентные',
    9 : 'либертарианские'    
    })

# Для языков создадим две переменные. Первая будет сообщать о том, сколько языков 
# Указано в профиле, вторая за то есть ли среди них английский 
engl = [ ]
lang_cnt = [ ]
for item in df_4.langs:
    try:
        engl.append(int('English' in item))
        lang_cnt.append(len(item))
    except:
        engl.append(None)
        lang_cnt.append(None)

df_4_clean['profile_engl_dummy'] = engl 
df_4_clean['profile_lang_cnt'] = lang_cnt 

# Время и платформу, с которой пользователь был в сети в последний раз 
# добавлять не будем, Мы соберем позже со стенок более крутые фичи

df_4_clean.head()

,uid,profile_albums_cnt,profile_audio_cnt,profile_followers_cnt,profile_friends_cnt,profile_gifts_cnt,profile_groups_cnt,profile_mutual_friends_cnt,profile_notes_cnt,profile_photos_cnt,...,profile_pages_cnt,profile_alco_love_cat,profile_smoke_love_cat,profile_religion_str,profile_inspired_by_str,profile_life_main_cat,profile_people_main_cat,profile_political_cat,profile_engl_dummy,profile_lang_cnt
0,22261760,1.0,1612.0,133.0,155.0,71.0,NaN,35.0,1.0,86.0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
1,295389188,0.0,0.0,53.0,198.0,NaN,NaN,8.0,0.0,42.0,...,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174168070,1.0,463.0,36.0,95.0,30.0,34.0,5.0,0.0,5.0,...,22.0,5.0,5.0,Буддизм,NaN,слава и влияние,власть и богатство,коммунистические,0.0,2.0
3,137312267,12.0,928.0,117.0,337.0,61.0,186.0,23.0,0.0,208.0,...,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47220748,0.0,0.0,47.0,113.0,NaN,NaN,1.0,17.0,8.0,...,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Объединим таблички по uid

In [32]:
print(df_4_clean.shape)
print(df_profile.shape)
df_profile = df_profile.merge(df_4_clean,left_on = 'uid',right_on = 'uid',how = 'outer')
print(df_profile.shape)

(706, 23)
(706, 31)
(706, 53)


#### 2.2.5 Пятая инфа

В пятой части инфы лежит ещё несколько списков. Посмотрим на них поподробнее. Это список школ, университетов и родственников.

In [33]:
infa[4][0]['response']

[{'first_name': 'Ксения',
  'last_name': 'Кулак',
  'occupation': {'id': 92,
   'name': 'РАНХиГС при Президенте РФ (АНХ при Правительстве РФ, РАГС при Президенте РФ)',
   'type': 'university'},
  'relatives': [{'type': 'sibling', 'uid': 50139669},
   {'type': 'sibling', 'uid': 141335795},
   {'type': 'sibling', 'uid': 31209146}],
  'schools': [{'city': 10280,
    'class': 'а',
    'country': 1,
    'id': '266806',
    'name': 'Школа №3',
    'year_from': 2010,
    'year_graduated': 2015,
    'year_to': 2015},
   {'city': 71,
    'class': 'а',
    'country': 1,
    'id': '11062',
    'name': 'Школа №18',
    'year_from': 2004,
    'year_to': 2009}],
  'uid': 22261760,
  'universities': [{'city': 1,
    'country': 1,
    'faculty': 6692,
    'faculty_name': 'Экономический факультет',
    'id': 92,
    'name': 'РАНХиГС при Президенте РФ (АНХ при Правительстве РФ, РАГС при Президенте РФ)'}]}]

In [34]:
# Очистим данные от респонсов.
inf_5 = [item['response'][0] for item in infa[4]]

Начнём потихоньку заполнять наш новый словарик, на основе которого будет создан пятый датафрейм.

1. У нас уже есть название вуза и название факультета в одном из предыдущих датасетов. Проигнорируем эту графу.
2. Школы. Преобразуем информацию о школах в несколько новых переменных. 
    * Количество школ, в которых учился человек
    * Буква класса в последней школе (не знаю зачем) 
    * В скольки городах находились школы этого человка
3. Список родственников. Часто в этих списках не очень родственники... Плюс они мало у кого заполнены. Вообще можно было бы вытащить все фамилии родственников и сверить являются ли они настоящими, но мы не будем. Слишком маленький датасет.

In [35]:
# В этот вектор будем помещать наши словарики 
dct_inf_5 = [ ]

for jtem in inf_5: 
    dct_5 = { }
    dct_5.update({'uid' : jtem['uid']})  # Добавляем id
    # Разбираемся со школами:
    try:  # Пробуем найти в списке школу.  Если получилось, заполняем информацию
        item = jtem['schools']
        if len(item) == 0:
            dct_5.update({'schools_count' : np.nan,
                           'last_bukva_class' : np.nan,
                           'change_city_school' : np.nan                          
                          })
        else:
            cts = set([it['city'] for it in item])
            dct_5.update({'schools_count' : len(item),
                            'last_bukva_class' : item[-1]['class'],
                            'change_city_school' : len(cts)  
                            })
    except: # Если не получилось, записываем пропуски. Не получится у людей, удаливших странички. 
        dct_5.update({'schools_count' : np.nan,
                       'last_bukva_class' : np.nan,
                        'change_city_school' : np.nan                          
                        })
    # Записываем новый элемент в вектор
    dct_inf_5.append(dct_5)   
    
df_5 = pd.DataFrame(dct_inf_5)
df_5.columns = ['profile_change_city_school_cnt', 'profile_last_bukva_class_str',
                'profile_schools_cnt', 'uid']
print(df_5.shape)
df_5.head()

(706, 4)


,profile_change_city_school_cnt,profile_last_bukva_class_str,profile_schools_cnt,uid
0,2.0,а,2.0,22261760
1,NaN,NaN,NaN,295389188
2,1.0,,1.0,174168070
3,NaN,NaN,NaN,137312267
4,NaN,NaN,NaN,47220748


Соединяем таблички. 

In [36]:
print(df_5.shape)
print(df_profile.shape)
df_profile = df_profile.merge(df_5,left_on = 'uid',right_on = 'uid',how = 'outer')
print(df_profile.shape)

(706, 4)
(706, 53)
(706, 56)


Пока что остановимся на этом и сохраним скачаную табличку. Не забываем послать себе в телеграм сообщение! 

In [38]:
df_profile.to_csv('vk_data_profile_'+version+'.csv',sep='\t')

!telegram-send "Сохранил таблицу с основной информацией. Приступаю к скачке особой инфы."

## 3. Фотографии

Скачаем ссылки на автарки всех людей. Аватарки это безумно важно. На их основе мы построим целую отдельную модель: свёрточную нейронную сеть, как бы страшно это не звучало. Под обработку аватарок в фичи мы пустим целый отдельный скрипт. 

In [39]:
vk_download('users.get','user_ids='+str(all_uids[0])+'&fields=crop_photo')['response']

[{'crop_photo': {'crop': {'x': 0.0, 'x2': 100.0, 'y': 0.0, 'y2': 100.0},
   'photo': {'aid': -6,
    'created': 1460398559,
    'height': 1221,
    'owner_id': 22261760,
    'pid': 409303847,
    'post_id': 3476,
    'src': 'https://pp.userapi.com/c629508/v629508760/3e431/0vgy3YlF7lc.jpg',
    'src_big': 'https://pp.userapi.com/c629508/v629508760/3e432/OfvbFgCeaPc.jpg',
    'src_small': 'https://pp.userapi.com/c629508/v629508760/3e430/u1A6StMdGwo.jpg',
    'src_xbig': 'https://pp.userapi.com/c629508/v629508760/3e433/aScmihfhaPM.jpg',
    'src_xxbig': 'https://pp.userapi.com/c629508/v629508760/3e434/SzEovsikyI8.jpg',
    'src_xxxbig': 'https://pp.userapi.com/c629508/v629508760/3e435/svI6QgGBu2I.jpg',
    'text': '',
    'width': 1221},
   'rect': {'x': 9.99, 'x2': 89.93, 'y': 9.99, 'y2': 89.93}},
  'first_name': 'Ксения',
  'last_name': 'Кулак',
  'uid': 22261760}]

Внутри нашего объекта crop_photo лежит два объекта: 

* photo - описание аватарки во всех подробностях. Нас интересуют лишь некоторые параметры.
    * created - дата добавления
    * height - высота фотки в пикселях
    * width - ширина фотки в пикселях 
    * src_XXX - url фотки с каким-то из размеров
    
* rect - по каким именно полям владелец страницы свою аватарку обрезал

Не очень понятно представляет ли для нас эта информация хоть какую-то ценность, но почему бы не забрать то, что нам дают.

In [40]:
dct_inf_photo = [ ]
k = 0  # Подсчитаем число неудач
# Идём по всем юзерам
for user in tqdm_notebook(all_uids):
    dct_cur = { }  # Информацию по каждому закидываем в словарь 
    dct_cur.update({'uid' : user}) # Не забываем запомнить id 
    time.sleep(0.4)
    pct = vk_download('users.get','user_id='+str(user)+'&fields=crop_photo')['response'][0]
    try:
        dct_cur.update({'ava_created': pct['crop_photo']['photo']['created']})
        dct_cur.update({'ava_height' : pct['crop_photo']['photo']['height']})
        dct_cur.update({'ava_width' : pct['crop_photo']['photo']['width']})
        # Заберём ссылку на самую большую аву...
        dct_cur.update({'ava_url' : pct['crop_photo']['photo']['src_big']})
        # Обрезка
        dct_cur.update({'ava_rect' : pct['crop_photo']['rect']})
    except: # Если не получилось, ставим везде None
        k += 1
        dct_cur.update({'ava_created' : None})
        dct_cur.update({'ava_height' : None})
        dct_cur.update({'ava_width' : None})
        # Заберём ссылку на самую большую аву...
        dct_cur.update({'ava_url' : None})
        # Обрезка
        dct_cur.update({'ava_rect' : None})        
    dct_inf_photo.append(dct_cur) 
# Смотрим на число ошибок
print(k)

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))


98


Около 60 человек не выставили аватарки вообще. В сумме с удаленными страницами получаем 96 безаватарных личностей. Сохраним фотки в словарик. На их обработку мы пустим отдельный скрипт. 

In [41]:
with open('vk_data_photos_' + version, 'wb') as f:
    pickle.dump(dct_inf_photo, f)
    
!telegram-send "Я скачал фотки"

## 4 Паблики, на которые подписан юзер

Отдельно вытащим паблики нашего юзера. С ними нам предстоит в будущем очень и очень серьёзно поработать. На это мы также пустим отдельный скрипт.

In [42]:
vk_download('users.getSubscriptions','user_ids='+str(uids[0]))['response']['groups']['items']

[25557243,
 33393308,
 111557607,
 135669953,
 24086381,
 20629724,
 131489096,
 158474183,
 11283947,
 98383010,
 102739356,
 63337812,
 18403220,
 75410445,
 155341956,
 129331635,
 41152133,
 150598112,
 42556983,
 91453124,
 77896376,
 148615008,
 54335419,
 65099143,
 41126705,
 69544687,
 104586508,
 67401618,
 76080856]

In [43]:
dct_inf_group = [ ]
k = 0 # Снова подсчитаем число неудач 
# Да и вообще весь код такой же как для аватарок
for user in tqdm_notebook(all_uids):
    dct_cur = { }
    dct_cur.update({'uid' : user})  # запоминаем id 
    time.sleep(0.4)
    try:
        grp = vk_download('users.getSubscriptions','user_id='+str(user))['response'] 
        dct_cur.update({'groups' : grp['groups']['items']}) # запоминаем список подписок 
    except:
        k += 1
        dct_cur.update({'groups' : np.nan}) # если подписок нет, запоминаем пропуск
    dct_inf_group.append(dct_cur)
print(k)

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))


34


Мы не смогли выудить подписки только у удалённых страниц. Судя по всему, код работает коректно. 

In [44]:
with open('vk_data_group_' + version, 'wb') as f:
    pickle.dump(dct_inf_group, f)
    
!telegram-send "Я скачал паблики юзера"

## 5. Стены юзеров 

Скачаем стены юзеров. Сохраним их в грязном виде,а затем обработаем и сохраним в чистом. 

In [45]:
# Функция для конкретного id собирает стену пользователя и выдает её в грязном виде
# и в виде таблицы с основной информацией о постах 

def Get_Wall(this_id):
    # Делай раз! Выясним сколько у пользователя на стене записей...
    n = vk_download('wall.get','owner_id=' + str(this_id))['response'][0]
    time.sleep(0.4) # После этого немного подождём
    
    # Качаем со стены все записи!
    chto_on_postit = [ ]
    for i in range(0,100*(n//100+1),100):
        # Качаем внеочередные 100 записей со стены.
        time.sleep(0.4) # Не забываем немного вздремнуть 
        wall = vk_download('wall.get', 'owner_id=' + str(this_id) +'&count=100&offset='+str(i))['response'][1:]
        # Первый элемент всегда число записей, игнорируем его 
        chto_on_postit.extend(wall)
        # chto_on_postit.extend([[item['id'],item['date']] for item in wall[1:]]) 
        
    # В файлике chto_on postit будут лежать грязные стенки.
    # Сразу же сварим важные фичи и запихнём их в табличку  
    # Подробнее: https://vk.com/dev/objects/post
    wi = { } # aka wall information 
    
    # Проходимся по информации из всех записей и собираем самое важное в фичи 
    for item in chto_on_postit:
        new_pos = [ ]
        # Поработаем с датой чуть более подробно. Преобразуем дату в удобочитаемый формат.
        real_data = datetime.datetime.fromtimestamp(item['date'])  
        new_pos.append(real_data)    # когда сделана запись
        new_pos.append(real_data.year)  # выделим год, когда сделана запись 
        new_pos.append(real_data.month) # выделим месяц
        new_pos.append(real_data.day)   # день, когда сделан пост
        new_pos.append(real_data.hour)  # час, когда сделан пост 
        
        # Нормальная дата для фичи "Когда сделан репостнутый пост"
        try:
            new_pos.append(datetime.datetime.fromtimestamp(item['copy_post_date']))
        except:
            new_pos.append(None)        
        
        new_pos.append(item['post_type'])             # тип записи post, copy, reply, postpone, suggest.
        new_pos.append(item['from_id'])               # кто оставил запись 
        new_pos.append(item['text'])                  # контент записи 
        new_pos.append(item['likes']['count'])        # число лайков 
        new_pos.append(item['reposts']['count'])      # число репостов 
        new_pos.append(item['comments']['count'])     # число комментов 
        new_pos.append(item['comments']['can_post'])  # можно ли оставлять комменты 
       
        # просмотры (если запись не старая), пропуск, если запись старая 
        try:
            new_pos.append(item['views']['count'])
        except:
            new_pos.append(None)
            
        # Истиный владелец записи (если это репост или запись оставлена)
        try:
            new_pos.append(item['copy_owner_id'])
        except:
            new_pos.append(None)   
            
        # Подробнее: https://vk.com/dev/objects/post_source
        # Через что создана запись 
        try:
            new_pos.append(item['post_source']['type'])
        except:
            new_pos.append(None)        
        
        # С какой платформы создана запись, если это телефон 
        try:
            new_pos.append(item['post_source']['platform'])
        except:
            new_pos.append(None)        
        
        # Дополнительный контент у поста
        try:
            new_pos.append([jtem['type'] for jtem in item['attachments']])
        except:
            new_pos.append(None)        
        # Сохраняям в словарик, ключ - id записи 
        wi[item['id']] = new_pos
        
    # превращаем всё, что насобирали в таблицу
    df = pd.DataFrame(wi).T
    df.columns = ['date', 'year', 'month', 'day','hour', 'copy_post_date', 'post_type', 'from_id', 'text', 
          'likes', 'reposts', 'comments', 'can_comment', 'views', 'copy_owner_id', 'type', 
          'platform', 'attachments']
    # добавляем столбец с id юзера
    df['uid'] = df.shape[0]*[this_id]
    return chto_on_postit, df

# Функция возвращает грязную стенку пользователя и обработанную таблчику 
# Грязную стену будем сохранять, а из таблички извлекать фичи 

In [46]:
# Проверяем работает ли 
cur_id = all_uids[32]
dirty_wall, df = Get_Wall(cur_id)
df.head()

,date,year,month,day,hour,copy_post_date,post_type,from_id,text,likes,reposts,comments,can_comment,views,copy_owner_id,type,platform,attachments,uid
480,2013-02-12 20:10:05,2013,2,12,20,2013-02-05 19:12:00,copy,49698902,Сегодня кто-то продавал на перекрестке счастье...,3,0,0,0,None,-38744784,vk,None,[photo],49698902
483,2013-02-17 22:44:12,2013,2,17,22,2013-02-17 16:04:00,copy,49698902,● БЫТЬ ОВНОМ:<br><br>Плюсы:<br>+ Всегда первые...,2,1,0,0,None,-40103104,vk,None,None,49698902
485,2013-02-17 23:04:38,2013,2,17,23,2013-02-13 10:11:34,copy,49698902,• ИМЕТЬ НЕОРДИНАРНОЕ МЫШЛЕНИЕ:<br><br>Плюсы:<b...,3,0,0,0,None,-40103104,vk,None,None,49698902
487,2013-03-29 11:37:27,2013,3,29,11,None,post,49698902,,62,0,0,0,None,None,vk,None,[photo],49698902
489,2013-03-29 23:31:41,2013,3,29,23,2013-03-29 23:30:00,copy,49698902,,6,1,0,0,None,-44759043,vk,None,[photo],49698902


Отлично! Всё работает. Начинаем варить фичи по нашим стеночкам. Какие-то из этих фичей мы чуть позже будем повторно обрабатывать и преобразовывать. Какие-то из них мы удалим при построении моделей из-за их малой информативности. Пока что вытащим информации по максималке. 

In [47]:
# Примечание: Если немного модернизировать этот кусок,  
# тогда можно будет варить фичи за какой-то период (год, месяц, квартал и тп)

def feature_extract(df):
    # Варим по табличке фичи за какой-то период 
    fd = { }
    fd['wall_post_cnt'] = df.shape[0]  # Всего записей на стене 
    # Фичи по колокам из типов постов и откуда сделаны посты 
    fd['wall_post_original_cnt'] = sum((df.post_type == 'post')&(df.from_id == cur_id)) # свои посты 
    fd['wall_post_other_cnt'] = sum((df.post_type == 'post')&(df.from_id != cur_id))    # чужие посты
    fd['wall_repost_group_cnt'] = sum((df.post_type == 'copy')&(df.copy_owner_id < 0))  # число репостов групп
    fd['wall_repost_user_cnt'] = sum((df.post_type == 'copy')&(df.copy_owner_id > 0))   # число репостов юзеров
    fd['wall_post_reply_cnt'] = sum(df.post_type == 'reply')        # ответы на своей стене на чужие записи

    # id тех, кого репостим (надо для детекции мусорных групп)
    fd['wall_repost_people_id'] = list(set(df.copy_owner_id[df.copy_owner_id > 0])) # люди
    fd['wall_repost_groups_id'] = list(set(df.copy_owner_id[df.copy_owner_id < 0])) # групры

    # Лайки, просмотры и тп
    fd['wall_like_cnt'] = df.likes.sum()                  # всего на стене лайков
    fd['wall_like_max'] = df.likes.max()                  # максимальное число лайков
    fd['wall_like_mean'] = df.likes.mean()                # среднее число лайков
    fd['wall_like_median'] = df.likes.median()            # медианное число лайков 

    fd['wall_repost_cnt'] = df.reposts.sum()              # всего репостов за стену 
    fd['wall_repost_max'] = df.reposts.max()              # максимальное число репостов
    fd['wall_repost_mean'] = df.reposts.mean()            # среднее число репостов
    fd['wall_repost_median'] = df.reposts.median()        # медианное число репостов

    fd['wall_can_comment'] = df.can_comment.get_values()[0] # можно ли коментить стенку 
    fd['wall_comment_cnt'] = df.comments.sum()            # всего комментов за стену 
    fd['wall_comment_max'] = df.comments.max()            # максимальное число коментов
    fd['wall_comment_mean'] = df.comments.mean()          # среднее число коментов
    fd['wall_comment_median'] = df.comments.median()      # медианное число коментов

    # С просмотрами всё плохо! Их забыли добавить в эту часть API 

    # Тип источника 
    fd['wall_post_instagram_cnt'] = sum((df.type == 'api')&(df.platform == 'instagram')) # посты из инстаграма
    fd['wall_post_android_cnt'] = sum((df.type == 'api')&(df.platform == 'android'))     # посты из андроида
    fd['wall_post_iphone_cnt'] = sum((df.type == 'api')&(df.platform == 'iphone'))       # посты с айфона
    fd['wall_post_ipad_cnt'] = sum((df.type == 'api')&(df.platform == 'ipad'))           # посты с апада
    fd['wall_post_wphone_cnt'] = sum((df.type == 'api')&(df.platform == 'wphone'))  # преступление против человечества

    fd['wall_post_web_cnt'] = sum(df.type == 'vk')          # запись через браузер 
    fd['wall_post_mweb_cnt'] = sum(df.type == 'mvk')        # запись через мобильный браузер 
    fd['wall_post_widget'] = sum(df.type == 'widget')  # запись создана через виджет на стороннем сайте обычно 
                                                       # это комменты с других сайтов где ты авторизован через вк

    # Немного информации по текстам записей 
    texts_len = [len(item) for item in df.text]
    fd['wall_text_len_cnt'] = np.sum(texts_len)            # суммарно символов на стене
    fd['wall_text_len_max'] = np.max(texts_len)            # максимально символов в посте
    fd['wall_text_len_mean'] = np.mean(texts_len)          # среднее число символов в посте
    fd['wall_text_len_median'] = np.median(texts_len)      # медианное число символов в посте 

    # Эмодзи-анализ записей 
    # Выбрасываем из записи все символы, кроме эмодзи и редких символов вроде иероглифов
    emoji = [re.sub('[a-яa-zA-ZА-Я -.,0-9<>:=;/_—@$^?–ё`%#*&!№…\|/\"\]\[\"\'\n\t”“’‘]','',item) for item in df.text]
    count_emoji = [len(item) for item in emoji]      # вычисляем эмодзи-след
    fd['wall_emoji_cnt'] = np.sum(count_emoji)       # суммарно эмодзи на стене 
    fd['wall_emoji_max'] = np.max(count_emoji)       # максимально эмодзи в посте
    fd['wall_emoji_mean'] = np.mean(count_emoji)     # среднее число эмодзи на стене 
    fd['wall_emoji_median'] = np.median(count_emoji) # медианное число эмодзи на стене 
    fd['wall_emoji_trace'] = ''.join(emoji)          # эмодзи-след человека
    fd['wall_text'] = ' '.join(df.text)              # весь текст целиком (надо для кластеров)

    # Дополнительный контент под постом 
    # Подробнее про контент https://vk.com/dev/objects/attachments_w
    content = ['photo', 'posted_photo', 'video', 'audio', 'doc', 'graffiti', 'link', 'note',
               'app', 'poll', 'page', 'album', 'photos_list', 'market', 'market_album', 'sticker']

    # Посчитаем по каждому типу вложений такие же метрики как выше и добавим в итоговый словарик
    def content_count(content_name):
        counter = [ ]
        for item in df.attachments:
            counter.append(np.sum(np.array(item) == content_name))
        return np.sum(counter), np.max(counter), np.mean(counter), np.median(counter)

    for cont in content:
        fd['wall_' + cont + '_cnt'], fd['wall_' + cont + '_max'], fd['wall_' + cont + '_mean'],\
               fd['wall_' + cont + '_median'] = content_count(cont)

    # Можно сделать ещё фич на основе времени, но лениво 
    fd['wall_yer_mean'] = np.mean([sum(df.year == item) for item in set(df.year)])     # В среднем за год постов 
    fd['wall_month_mean'] = np.mean([sum(df.month == item) for item in set(df.month)]) # В среднем за месяц постов 

    # Сколько времени проходит между тем как автор сделал пост и человек это репостнул 
    df_copy = df[df.post_type == 'copy']
    slow = [ss.seconds/3600 for ss in  df_copy.date - df_copy.copy_post_date if ss.days < 500]
    # Условие < 500 нужно, чтобы отсеять выбросы. Иногда дата стоит некоректно 
    if len(slow) > 0:
        fd['wall_slowpok_hour_median'] = np.median(slow)
        fd['wall_slowpok_hour_max'] = np.max(slow)
    else:
        fd['wall_slowpok_hour_median'] = 0 
        fd['wall_slowpok_hour_max'] = 0
        
    fd['uid'] = df.uid.get_values()[0]
    return fd     

Самая сложная часть осталась позади. Осталось извлечь! Теперь идём по каждому id из заготовленного выше списка, вытаскиваем стену этого человека, сохраняем её в грязном виде, после изготавливаем фичи и их тоже сохраняем. 

In [48]:
%%time
dwd = { }       # aka dirty wall dict
wall_infa = { } # Табличка с информацией по стенкам 
dirty_df_list = [ ] # Сюда будем записывать грязные таблички
k = 0           # Счётчик для выяснения сколько страниц удалены 
l = 1           # Индекс для таблички 
for uid in tqdm_notebook(all_uids):
    l += 1
    time.sleep(0.4) # И пусть весь мир подождёт! 
    try:
        dirty_wall, df = Get_Wall(uid)
        dirty_df_list.append(df)
        dwd[uid] = dirty_wall
        wall_infa[l] =  feature_extract(df)
    except:
        wall_infa[l] = {'uid': uid} # работает ли такое говно 
        k += 1
print(k)

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))


82
CPU times: user 2min 53s, sys: 7.35 s, total: 3min
Wall time: 1h 1min 53s


Количество людей без стены сопостовимо с количеством людей без фоток (увидим это ниже). Ох уж эти мезантропы... Сохраняем данные. Откровенно говоря, 110 фичей это очень много для нашей небольшой выборки. Многое придётся выбросить. 

In [49]:
df_wall = pd.DataFrame(wall_infa).T
print(df_wall.shape)
df_wall.head()

(706, 108)


,uid,wall_album_cnt,wall_album_max,wall_album_mean,wall_album_median,wall_app_cnt,wall_app_max,wall_app_mean,wall_app_median,wall_audio_cnt,...,wall_text,wall_text_len_cnt,wall_text_len_max,wall_text_len_mean,wall_text_len_median,wall_video_cnt,wall_video_max,wall_video_mean,wall_video_median,wall_yer_mean
2,22261760,0,0,0,0,0,0,0,0,39,...,"ох уж эти воспоминания, я вам так скажу классн...",6101,830,61.6263,17,8,7,0.0808081,0,16.5
3,295389188,0,0,0,0,0,0,0,0,7,...,Шикарный стих. Если он тебе понравился - д...,4303,3961,172.12,0,0,0,0,0,8.33333
4,174168070,0,0,0,0,0,0,0,0,0,...,,0,0,0,0,0,0,0,0,1
5,137312267,0,0,0,0,0,0,0,0,93,...,Я оценила твою фотографию в приложении ТопФе...,25370,2588,74.3988,45,16,5,0.0469208,0,48.7143
6,47220748,0,0,0,0,0,0,0,0,1,...,#ночник_slvp<br><br>По пустым коридорам школы...,5771,5771,1923.67,0,1,1,0.333333,0,1


In [50]:
# Чистые стены
df_wall.to_csv('vk_wall_data_'+ version + '.csv',sep='\t')

# Грязные стены 
with open('vk_dirty_wall_' + version, 'wb') as f:
    pickle.dump(dwd, f)

На всякие разные штуки вроде эмодзи-следов и того из каких именно пабликов люди делают репосты мы также посмотрим в отдельном скрипте, который будет посвящён работе с группами. 

На всякий случай для других исследований сохраним грязную супертаблицу. 

In [51]:
super_dirty_df = pd.concat(dirty_df_list)
print(super_dirty_df.shape)
super_dirty_df.head( )

(275364, 19)


,date,year,month,day,hour,copy_post_date,post_type,from_id,text,likes,reposts,comments,can_comment,views,copy_owner_id,type,platform,attachments,uid
1918,2011-09-08 21:53:21,2011,9,8,21,2011-09-08 21:07:16,copy,22261760,"ох уж эти воспоминания, я вам так скажу",11,0,0,1,None,-21193611,vk,None,None,22261760
2762,2012-09-19 19:43:33,2012,9,19,19,None,post,141335795,классно сделали!,1,0,4,1,None,None,vk,None,None,22261760
2767,2012-09-24 19:09:00,2012,9,24,19,2012-09-20 22:03:00,copy,22261760,...прогуляться по узким итальянским улочкам,8,2,0,1,None,-36008740,vk,None,None,22261760
2768,2012-09-24 21:14:42,2012,9,24,21,2012-09-24 21:03:04,copy,22261760,...в Италию,6,0,5,1,None,-36008740,vk,None,None,22261760
2774,2012-09-26 21:42:24,2012,9,26,21,None,post,152405463,С ДНЁМ РОЖДЕНИЯ!!!,1,0,4,1,None,None,vk,None,[photo],22261760


In [52]:
super_dirty_df.to_csv('vk_dirty_wall_data'+ version + '.csv', sep='\t')

## 6. Неаватарные фотки юзеров 

Скачаем абсолютно все ссыки на фотки юзеров. Гулять, nак по крупному! Всё сопутствующее тоже будем нагло скачивать. Делать это будем один в один как со стенами. 

In [53]:
vk_download('photos.getAll','owner_id='+ str(all_uids[0]) +'&no_service_albums=0&extended=1'+\
                               '&count=100')['response'][10]

{'aid': -6,
 'created': 1372011341,
 'height': 1707,
 'likes': {'count': 33, 'user_likes': 0},
 'owner_id': 22261760,
 'pid': 304399783,
 'post_id': 3103,
 'reposts': {'count': 0},
 'src': 'https://pp.userapi.com/c425523/v425523760/1187/HLJNKTmcZYg.jpg',
 'src_big': 'https://pp.userapi.com/c425523/v425523760/1188/o0rtSFGA8p4.jpg',
 'src_small': 'https://pp.userapi.com/c425523/v425523760/1186/qWlE4beBhXo.jpg',
 'src_xbig': 'https://pp.userapi.com/c425523/v425523760/1189/Xi_YcReMxrs.jpg',
 'src_xxbig': 'https://pp.userapi.com/c425523/v425523760/118a/xymMdgyIilQ.jpg',
 'src_xxxbig': 'https://pp.userapi.com/c425523/v425523760/118b/xWkgXfnDoiw.jpg',
 'text': '',
 'width': 2560}

In [54]:
# Функция для конкретного id собирает фоточки пользователя и выдает их в грязном виде
# и в виде таблицы с основной информацией о постах. Также на выход выдаётся словарь из ссылок на фотки,
# в котором ключами являются id фоток 

# Функция будет пытаться добавить в вектор из контактовского ответа 
# новый элемент, если его там нет, то добавит None 
def tr_ex(add, out, name):
    try:
        add.append(out[name])
    except:
        add.append(None)

# То же самое, но для двууровнегого словоря 
# (знаю, звучит сложно, но это не так)
def tr_ex2(add, out, name1, name2):
    try:
        add.append(out[name1][name2])
    except:
        add.append(None)
        

def Get_Photo(this_id):
    # Делай раз! Выясним сколько у пользователя фоток...
    n = vk_download('photos.getAll','owner_id='+ str(this_id))['response'][0]
    time.sleep(0.4) # Сразу же после этого немного подождём
    
    # Качаем все фотки из альбомов!
    chto_on_fotkaet = [ ]
    for i in range(0,100*(n//100+1),100):
        #Качаем внеочередные 100 фоточек
        time.sleep(0.4) # Не забываем немного подождать перед каждой итераций
        photos = vk_download('photos.getAll','owner_id='+ str(this_id) +'&no_service_albums=0&extended=1'+\
                               '&count=100&offset='+str(i))['response'][1:]
        # Первый ээлемент всегда число фоток, игнорируем его
        chto_on_fotkaet.extend(photos)
        
    # В файлике chto_on_fotkaet будут лежать грязные фотки.
    # Сразу же сварим важные фичи и запихнём их в табличку  
    # Подробнее: https://vk.com/dev/objects/photo
    phi = { } # aka photo information 
    
    # Проходимся по информации из всех фоток и забираем информацию
    for item in chto_on_fotkaet:
        new_ph = [ ]   # Вектор для наблюдений 
        
        # Делай раз! Дата. Преобразуем её в читаемый вид 
        real_data = datetime.datetime.fromtimestamp(item['created'])  
        new_ph.append(real_data)       # когда залита фотка
        new_ph.append(real_data.year)  # выделим год, когда залита фотка
        new_ph.append(real_data.month) # выделим месяц
        new_ph.append(real_data.day)   # день
        new_ph.append(real_data.hour)  # час
        
        tr_ex(new_ph, item, 'width')   # ширина фотки
        tr_ex(new_ph, item, 'height')  # высота фотки

        new_ph.append(item['owner_id']) # id владельца
        new_ph.append(item['pid'])      # id фоточки 
        tr_ex(new_ph, item, 'aid')      # id альбома (для поиска авок)
        
        new_ph.append(item['text'])     # Текст под фоткой
        
        # Пытаемся добавить ссылки на фотки всех возможных размеров. 
        tr_ex(new_ph, item, 'src_small')
        tr_ex(new_ph, item, 'src')
        tr_ex(new_ph, item, 'src_big')
        tr_ex(new_ph, item, 'src_xbig')
        tr_ex(new_ph, item, 'src_xxbig')
        tr_ex(new_ph, item, 'src_xxxbig')
        
        # Собираем количество лайков под фоткой 
        tr_ex2(new_ph, item, 'likes', 'count')
        # Собираем количество репостов под фоткой 
        tr_ex2(new_ph, item, 'reposts', 'count')
        # Собираем количество комментов под фоткой 
        tr_ex2(new_ph, item, 'comments', 'count')        
        # Собираем количество тэгов под фоткой 
        tr_ex2(new_ph, item, 'tags', 'count')
        
        # Закидываем это чудо в словарик 
        phi[item['pid']] = new_ph
    
    # Превращаем всё, что насобирали в табличку 
    df = pd.DataFrame(phi).T
    df.columns = ['data', 'year', 'month', 'day', 'hour', 'width', 'height', 'uid',
                 'pid', 'aid', 'text', 'src_small', 'src', 'src_big', 'src_xbig', 'src_xxbig',
                 'src_xxxbig','like','repost','comment','tags']
    return chto_on_fotkaet, df

# Проверяем работает ли 
ch1, df1 = Get_Photo(all_uids[0])
df1.tail() 

,data,year,month,day,hour,width,height,uid,pid,aid,...,src_small,src,src_big,src_xbig,src_xxbig,src_xxxbig,like,repost,comment,tags
350583246,2015-01-19 20:54:53,2015,1,19,20,960,960,22261760,350583246,-6,...,https://pp.userapi.com/c625116/v625116760/16d1...,https://pp.userapi.com/c625116/v625116760/16d1...,https://pp.userapi.com/c625116/v625116760/16d1...,https://pp.userapi.com/c625116/v625116760/16d1...,https://pp.userapi.com/c625116/v625116760/16d1...,None,53,0,None,None
366274765,2015-05-23 08:10:17,2015,5,23,8,960,640,22261760,366274765,-7,...,https://pp.userapi.com/c629407/v629407760/2c/X...,https://pp.userapi.com/c629407/v629407760/2d/D...,https://pp.userapi.com/c629407/v629407760/2e/J...,https://pp.userapi.com/c629407/v629407760/2f/0...,https://pp.userapi.com/c629407/v629407760/30/C...,None,20,0,None,None
377987282,2015-09-01 22:54:43,2015,9,1,22,878,720,22261760,377987282,-7,...,https://pp.userapi.com/c622126/v622126760/3f9d...,https://pp.userapi.com/c622126/v622126760/3f9d...,https://pp.userapi.com/c622126/v622126760/3f9d...,https://pp.userapi.com/c622126/v622126760/3f9d...,https://pp.userapi.com/c622126/v622126760/3f9d...,None,6,0,None,None
377987283,2015-09-01 22:54:44,2015,9,1,22,960,960,22261760,377987283,-7,...,https://pp.userapi.com/c622126/v622126760/3f9d...,https://pp.userapi.com/c622126/v622126760/3f9e...,https://pp.userapi.com/c622126/v622126760/3f9e...,https://pp.userapi.com/c622126/v622126760/3f9e...,https://pp.userapi.com/c622126/v622126760/3f9e...,None,13,0,None,None
409303847,2016-04-11 21:15:59,2016,4,11,21,1221,1221,22261760,409303847,-6,...,https://pp.userapi.com/c629508/v629508760/3e43...,https://pp.userapi.com/c629508/v629508760/3e43...,https://pp.userapi.com/c629508/v629508760/3e43...,https://pp.userapi.com/c629508/v629508760/3e43...,https://pp.userapi.com/c629508/v629508760/3e43...,https://pp.userapi.com/c629508/v629508760/3e43...,81,0,None,None


Ура! Снова работает. Создаём из этого всего дела фичи. 

In [55]:
# Функция для поиска ссылок в тексте
url_start_1 = 'http[s]?://'
url_start_2 = 'vk'
url_end = '(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
pattern = url_start_1 + url_end + '|' + url_start_2 + url_end

def get_clean_url(url):
    return re.findall(pattern, url)

# Создаем фичи из скаченой таблички с фоточками
def feature_extract_ph(df):
    # Варим фичи по табличке с фоточками
    fd = { }
    fd['photo_cnt'] = df.shape[0] # Всего фото в несистемных альбомах (вроде уже есть така фича)
    
    # Как часто человек меняет аватар 
    fd['vk_photo_ava_change_cnt'] = df[df.aid == -6].shape[0]
    # Как часто постит фотки на стенку 
    fd['vk_photo_wall_ph_post_cnt'] = df[df.aid == -7].shape[0]
    
    # Можно сделать ещё фич на основе времени, но лениво 
    fd['photo_yer_mean'] = np.mean([sum(df.year == item) for item in set(df.year)])     # В среднем за год постов 
    fd['photo_month_mean'] = np.mean([sum(df.month == item) for item in set(df.month)]) # В среднем за месяц постов 
    
    # Высота и ширина фоток (скорее всего это бесполезные фичи)
    fd['photo_width_mean'] = df.height.dropna().mean()
    fd['photo_height_mean'] = df.width.dropna().mean()
    
    # Лайки, комменты и тп
    fd['photo_like_cnt'] = df.like.dropna().sum()              # всего лайков под фотами
    fd['photo_like_max'] = df.like.dropna().max()              # максимальное число лайков
    fd['photo_like_mean'] = df.like.dropna().mean()            # среднее число лайков
    fd['photo_like_median'] = df.like.dropna().median()        # медианное число лайков 
    
    fd['photo_repost_cnt'] = df.repost.dropna().sum()          # всего лайков под фотами
    fd['photo_repost_max'] = df.repost.dropna().max()          # максимальное число лайков
    fd['photo_repost_mean'] = df.repost.dropna().mean()        # среднее число лайков
    fd['photo_repost_median'] = df.repost.dropna().median()    # медианное число лайков 
    
    # Комменты и тэги есть в API, но их нет в реквестах 
    # fd['photo_comment_cnt'] = df.comment.dropna().sum()         # всего лайков под фотами
    # fd['photo_comment_max'] = df.comment.dropna().max()         # максимальное число лайков
    # fd['photo_comment_mean'] = df.comment.dropna().mean()       # среднее число лайков
    # fd['photo_comment_median'] = df.comment.dropna().median()   # медианное число лайков 
    
    # fd['photo_tags_cnt'] = df.tags.dropna().sum()               # всего лайков под фотами
    # fd['photo_tags_max'] = df.tags.dropna().max()               # максимальное число лайков
    # fd['photo_tags_mean'] = df.tags.dropna().mean()             # среднее число лайков
    # fd['photo_tags_median'] = df.tags.dropna().median()         # медианное число лайков 

    # Немного информации по подписям к фоточкам
    texts_len = [len(item) for item in df.text]
    fd['photo_text_len_cnt'] = np.sum(texts_len)                # суммарно символов в подписях 
    fd['photo_text_len_cnt'] = np.max(texts_len)                # максимально символов в подписях 
    fd['photo_text_len_cnt'] = np.mean(texts_len)               # среднее число символов в подписях 
    fd['photo_text_len_cnt'] = np.median(texts_len)             # медианное число символов в подписях 
    
    a = ' '.join(df.text)                        
    fd['vk_photo_text_url_len_cnt'] = len(get_clean_url(a))        # число ссылок в тексте 
    fd['photo_text'] = a                                           # весь текст целиком
    
    # id владельца аккаунта 
    fd['uid'] = df.uid.get_values()[0]
    return fd   

# Проверяем работает ли 
feature_extract_ph(df1)

{'photo_cnt': 86,
 'photo_height_mean': 859.14814814814815,
 'photo_like_cnt': 1142,
 'photo_like_max': 81,
 'photo_like_mean': 13.279069767441861,
 'photo_like_median': 10.5,
 'photo_month_mean': 7.166666666666667,
 'photo_repost_cnt': 0,
 'photo_repost_max': 0,
 'photo_repost_mean': 0.0,
 'photo_repost_median': 0.0,
 'photo_text': '           Создано в приложение ФотоСтатус - vkontakte.ru/app2175066?from_id=1&amp;loc=4d85954f7ee0d88d0ec43360 Создано в приложение ФотоСтатус - vkontakte.ru/app2175066?from_id=1&amp;loc=4d85954f7ee0d88d0ec43360 Создано в приложение ФотоСтатус - vkontakte.ru/app2175066?from_id=1&amp;loc=4d85954f7ee0d88d0ec43360 Создано в приложение ФотоСтатус - vkontakte.ru/app2175066?from_id=1&amp;loc=4d85954f7ee0d88d0ec43360 Создано в приложение ФотоСтатус - vkontakte.ru/app2175066?from_id=1&amp;loc=4d85954f7ee0d88d0ec43360   Создано в приложение ФотоСтатус - vkontakte.ru/app2175066?from_id=1&amp;loc=4d8614927ee0d88d134a3d60 Создано в приложение ФотоСтатус - vkontakte.r

Готово! Проходимя по каждому id. 

In [56]:
%%time
dphd = { }       # aka dirty photo dict
ph_infa = { }    # словарик с информацией по фоткам
dirty_df_list = [ ] # Сюда будем записывать грязные таблички
k = 0           # Счётчик для выяснения сколько страниц удалены 
l = 1           # Индекс для таблички 

for uid in tqdm_notebook(all_uids):
    l += 1
    time.sleep(0.4) # И пусть весь мир подождёт! 
    try:
        dirty_ph, df = Get_Photo(uid)
        dirty_df_list.append(df)
        dphd[uid] = dirty_ph
        ph_infa[l] =  feature_extract_ph(df)
    except:
        ph_infa[l] = {'uid': uid} # работает ли такое говно 
        k += 1

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))


CPU times: user 49.3 s, sys: 2.68 s, total: 52 s
Wall time: 24min 28s


In [57]:
print(k)
df_ph = pd.DataFrame(ph_infa).T
print(df_ph.shape)
df_ph.head()

70
(706, 19)


,photo_cnt,photo_height_mean,photo_like_cnt,photo_like_max,photo_like_mean,photo_like_median,photo_month_mean,photo_repost_cnt,photo_repost_max,photo_repost_mean,photo_repost_median,photo_text,photo_text_len_cnt,photo_width_mean,photo_yer_mean,uid,vk_photo_ava_change_cnt,vk_photo_text_url_len_cnt,vk_photo_wall_ph_post_cnt
2,86,859.148,1142,81,13.2791,10.5,7.16667,0,0,0,0,Создано в приложение ФотоСтатус - v...,0,847.148,12.2857,22261760,47,23,16
3,42,946.286,974,201,23.1905,14,4.2,1,1,0.0238095,0,,0,925.476,14,295389188,18,0,24
4,5,960,94,28,18.8,22,1.25,0,0,0,0,,0,1036.8,1.66667,174168070,2,0,3
5,208,1217.96,6192,243,29.7692,20,17.3333,0,0,0,0,лето** Мамуля ...) ...,0,1339.53,29.7143,137312267,34,0,0
6,8,709,12,8,1.5,0.5,1.6,0,0,0,0,Я выиграл партию в монополию vk.com/magna...,0,435.875,2,47220748,0,1,8


In [58]:
# Чистые фотки
df_ph.to_csv('vk_photo_data_' + version + '.csv',sep='\t')

# Грязные фотки
with open('vk_dirty_photo_' + version, 'wb') as f:
    pickle.dump(dphd, f)

Сохраняем огромную табличку с перечнем фоток. Она нам ещё пригодится в будущем. 

In [59]:
super_dirty_df = pd.concat(dirty_df_list)
print(super_dirty_df.shape)
super_dirty_df.head( )

(60882, 21)


,data,year,month,day,hour,width,height,uid,pid,aid,...,src_small,src,src_big,src_xbig,src_xxbig,src_xxxbig,like,repost,comment,tags
190267459,2010-11-10 14:41:38,2010,11,10,14,None,None,22261760,190267459,-6,...,https://pp.userapi.com/c11497/u22261760/-6/s_5...,https://pp.userapi.com/c11497/u22261760/-6/m_8...,https://pp.userapi.com/c11497/u22261760/-6/x_b...,None,None,None,10,0,None,None
190333944,2010-11-10 20:56:08,2010,11,10,20,None,None,22261760,190333944,-6,...,https://pp.userapi.com/c11497/u22261760/-6/s_4...,https://pp.userapi.com/c11497/u22261760/-6/m_4...,https://pp.userapi.com/c11497/u22261760/-6/x_6...,None,None,None,8,0,None,None
192922595,2010-11-26 14:59:51,2010,11,26,14,None,None,22261760,192922595,-6,...,https://pp.userapi.com/c9595/u22261760/-6/s_be...,https://pp.userapi.com/c9595/u22261760/-6/m_8b...,https://pp.userapi.com/c9595/u22261760/-6/x_8c...,None,None,None,4,0,None,None
199181003,2010-12-17 19:08:04,2010,12,17,19,None,None,22261760,199181003,-6,...,https://pp.userapi.com/c4454/u22261760/-6/s_1d...,https://pp.userapi.com/c4454/u22261760/-6/m_e9...,https://pp.userapi.com/c4454/u22261760/-6/x_f6...,None,None,None,7,0,None,None
212021276,2011-01-19 21:07:52,2011,1,19,21,None,None,22261760,212021276,-6,...,https://pp.userapi.com/c9689/u22261760/-6/s_19...,https://pp.userapi.com/c9689/u22261760/-6/m_c9...,https://pp.userapi.com/c9689/u22261760/-6/x_61...,None,None,None,18,0,None,None


In [61]:
super_dirty_df.to_csv('vk_dirty_photo_data' + version + '.csv', sep='\t')

## 7. Лайки

Наконец мы дошли до самой долгой части кода. До лайков. Соберём лайки, которые были поставлены каждому юзеру под фотки и на стенку. После мы сможем посмотреть на то как именно перетекают лайки в рамках иканамовского множества юзеров. 

In [63]:
# Подгрузим грязную стену и грязные фотки 
with open('vk_dirty_wall_' + version, 'rb') as f:
    dwd = pickle.load(f)

with open('vk_dirty_photo_' + version, 'rb') as f:
    dphd = pickle.load(f)

In [64]:
this_uid = all_uids[0]
wall = [[item['id'],item['date']] for item in dwd[this_uid]]
wall[:5]

[[3486, 1474943934],
 [3468, 1459243690],
 [3464, 1447501613],
 [3460, 1443369796],
 [3458, 1443346510]]

Чтобы скачать все лайки можно написать функцию, которая написана ниже. Она будет делать один запрос, вытаскивать лайки и радоваться жизни. 

In [65]:
# Скачаем с уже скаченой стены всю информацию о лайках 
# Создадим у функции необязательный аргумент - дата конца выкачки.
def get_likes(this_uid, dwd, end_date = date.today().replace(year = date.today().year - 2)):
    # Распаковываем стену
    wall = [[item['id'],item['date']] for item in dwd[this_uid]]
    # Качаем пользователей, лайкнувших записи на cтене юзера за год!
    wall_likes_u = { }
    for item in wall:
        # Немножечко подождём, а потом скачаем пост.
        time.sleep(0.4)
        a = vk_download('likes.getList','type=post&owner_id='+str(this_uid)+'&item_id='+str(item[0])+'&count=1000')
        # преобразуем время
        normal_time = datetime.datetime.fromtimestamp(item[1])
        # запишем в словарь новое наблюдение
        wall_likes_u.update({normal_time.strftime('%Y-%m-%d %H:%M:%S'): a['response']['users']})
        # Если мы пробиваем границу снизу величиной с год, останавливаем сбор данных:
        if normal_time.date() < end_date:
            break
    return wall_likes_u

In [66]:
date.today().replace(year = date.today().year - 2)

datetime.date(2015, 12, 30)

In [67]:
get_likes(all_uids[0], dwd)

{'2015-11-14 14:46:53': [13113950,
  37869736,
  50139669,
  73614108,
  89784122,
  124885357,
  132022070,
  133995592,
  145819622,
  155206444,
  159365241,
  212710486,
  217819228,
  220963201,
  227558867,
  265755171,
  347797061],
 '2016-03-29 12:28:10': [16622554,
  50139669,
  79473594,
  89784122,
  107749291,
  132035102,
  141335795,
  156619631,
  159365241,
  191335615,
  202696404,
  232276410,
  242183537,
  289729652,
  322723607,
  347373543,
  399045384],
 '2016-09-27 05:38:54': [454460773,
  50139669,
  89784122,
  132035102,
  145819622,
  202696404,
  227558867,
  265772903,
  399045384,
  408027254,
  415740262]}

Вроде бы всё работает. Теперь нам остаётся лишь пробежаться циклом по всем постам и всё. Такой цикл будет работать около 10 часов. И это довольно долго. Именно поэтому, мы не будем запускать этот код. 

In [ ]:
wall_likes = { }
for uid in tqdm_notebook(all_uids):
    try:
        wall_likes[uid] = get_likes(uid, dwd)
    except:
        print('Возникли проблемы с ', uid)
        
len(wall_likes)

In [ ]:
# Сохраняем накачаное добро 
with open('vk_total_wal_likes_' + version, 'wb') as f:
    pickle.dump(wall_likes, f)

После можно обновить токен, перезапустить функцию `vk_download`, так как суммарно пройдёт уже около 24 часов работы кода, а токен предоставляет доступ ровно на 24 часа, а после повторить те же манипуляции с лайками под фотками. Для этого надо немного изменить функцию выше. 

Этот кусок кода отлично отработает и всё будет круто. Однако это всё слишком долго. К счастью, процесс можно ускорить с помощью метода [execute.](https://vk.com/dev/execute) Этот метод позволяет делать цепи из параллельных запросов к вк. Максимально в один батч из запросов можно поместить 25 запросов. Вы уже читали о нём в туториале и сейчас пришло время использовать его в боевых условиях. 

Напомню, что метод работает следующим образом. Мы должны перечислить через запятую внутри нашей ссылки все 25 запросов, которые мы делаем. И тогда в качестве ответа мы получим вектор из лайков. 

In [149]:
url = 'https://api.vk.com/method/execute?code=return[\
API.likes.getList({"type":"post","owner_id":"22261760","item_id":"3468","count":"1000"}),\
API.likes.getList({"type":"post","owner_id":"22261760","item_id":"3486","count":"1000"})];&access_token='+token

res = requests.get(url)
res.json( )

{'response': [{'count': 17,
   'users': [16622554,
    50139669,
    79473594,
    89784122,
    107749291,
    132035102,
    141335795,
    156619631,
    159365241,
    191335615,
    202696404,
    232276410,
    242183537,
    289729652,
    322723607,
    347373543,
    399045384]},
  {'count': 11,
   'users': [454460773,
    50139669,
    89784122,
    132035102,
    145819622,
    202696404,
    227558867,
    265772903,
    399045384,
    408027254,
    415740262]}]}

Например, в данной ситуации, мы попросили его отдать там юзеров, которые лайкнули два конкретных поста и получили их. Итак, перейдэм к делу. Напишем функцию, которая для каждого конкретного пользователя будет понимать сколько у него на стене постов и будет делать из них батчи по 25 штук. Будем делать это для глубины в два года. 

In [151]:
def make_batch(this_uid, dwd, end_date):
    # Делаем массив вида [id, дата]
    # В условии отбираем все даты, которые младще 2 лет
    wall = [[item['id'],item['date']] for item in dwd[this_uid] if datetime.datetime.fromtimestamp(item['date']).date() > end_date ]
    # Делим стенку на батчи 
    batches = [wall[i:i+25] for i in range(0,len(wall),25)]
    return batches 


In [152]:
[len(item) for item in make_batch(all_uids[570], dwd, date.today().replace(year = date.today().year - 2))]

[25, 25, 25, 14]

Вроде бы всё работает. Теперь напишем функцию, которая для батча будет генерировать запрос в виде ссылки. 

In [153]:
def make_url(this_uid, batch, token):
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token
    s = ''
    for bt in batch:
        s += 'API.likes.getList({"type":"post","owner_id":"' + str(this_uid) + \
             '","item_id":"' + str(bt[0]) + '","count":"1000"}),'
    return begin + s[:-1] + end 

In [154]:
url = make_url(all_uids[570], make_bathc(all_uids[570])[-1], token)
url

'https://api.vk.com/method/execute?code=return[API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1020","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1019","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1018","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1017","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1016","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1015","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1014","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1013","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1012","count":"1000"}),API.likes.getList({"type":"post","owner_id":"123854432","item_id":"1011","count":"1000"}),API.likes.getList({"type":"post","owner_id":"12385443

In [155]:
res = requests.get(url)
itg = res.json( )['response']
print(len(itg))
itg[1]

14


{'count': 30,
 'users': [21097773,
  27099012,
  38329697,
  62732507,
  83445093,
  92735376,
  143781440,
  172330725,
  176129447,
  190757255,
  201070708,
  225064096,
  288111286,
  315032988,
  339907557,
  374733634,
  375598448,
  376944969,
  378250991,
  378790107,
  379993680,
  380183622,
  381326478,
  382113370,
  382260549,
  395254172,
  399493958,
  414087916,
  414098928,
  449477956]}

Получаем за один запрос дофига ответов. Всё работает. Пишем итоговую функцию. 

In [178]:
def get_likes(this_uid, dwd=dwd, end_date = date.today().replace(year = date.today().year - 2), token = token):
    # Создаём под лайки вектор 
    wall_likes_u = { }
    # Делаем батчи
    batch = make_batch(this_uid, dwd, end_date)
    for btc in batch:
        # Создаём для батча ссылку 
        url = make_url(this_uid, btc, token)
        # Делаем запрос
        time.sleep(0.4)
        res = requests.get(url)
        itg = res.json( )['response']
        # Распаковываем в формате {время поста: юзеры, которые лайкали}
        wall_likes_u.update({datetime.datetime.fromtimestamp(b[1]).strftime('%Y-%m-%d %H:%M:%S') 
                             : it['users'] for b,it in zip(btc,itg)})
    return wall_likes_u

In [181]:
loises = get_likes(all_uids[570])
loises['2016-01-20 06:50:47']

[4345637,
 68402525,
 122988266,
 146184209,
 163285645,
 285812069,
 288111286,
 328620604,
 384078627,
 449477956]

Ура! Всё работает. Теперь мы можем задействовать нашу функцию в промышленных масштабах. Пускаем цикл по всем пользователям. 

In [182]:
wall_likes = { }
for uid in tqdm_notebook(all_uids):
    try:
        wall_likes[uid] = get_likes(uid)
    except:
        print('Возникли проблемы с ', uid)

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))

Возникли проблемы с  266510396
Возникли проблемы с  28317794
Возникли проблемы с  52144231
Возникли проблемы с  422131817
Возникли проблемы с  333801583
Возникли проблемы с  235595930
Возникли проблемы с  263618776
Возникли проблемы с  4751578
Возникли проблемы с  5058781
Возникли проблемы с  49289444
Возникли проблемы с  137859376
Возникли проблемы с  321378609
Возникли проблемы с  65581359
Возникли проблемы с  27742513
Возникли проблемы с  156797260
Возникли проблемы с  124770636
Возникли проблемы с  123857237
Возникли проблемы с  248191351
Возникли проблемы с  143817090
Возникли проблемы с  172437917
Возникли проблемы с  322136507
Возникли проблемы с  247298498
Возникли проблемы с  43481543
Возникли проблемы с  6521301
Возникли проблемы с  224768482
Возникли проблемы с  137320994
Возникли проблемы с  66116161
Возникли проблемы с  6931026
Возникли проблемы с  26579546
Возникли проблемы с  149054062
Возникли проблемы с  63482547
Возникли проблемы с  5188414
Возникли проблемы с  737431

In [183]:
len(wall_likes)

624

In [189]:
# Сохраняем накачаное добро 
with open('vk_total_wal_likes_' + version, 'wb') as f:
    pickle.dump(wall_likes, f)

Время работы кода в разы уменьшилось. Для того, чтобы по аналогии сделать с фотками, достаточно просто немного переписать порождателя ссылок. 

In [193]:
#Поменяем id на pid, date на created
def make_batch(this_uid, dwd, end_date):
    # Делаем массив вида [id, дата]
    # В условии отбираем все даты, которые младще 2 лет
    wall = [[item['pid'],item['created']] for item in dwd[this_uid] if datetime.datetime.fromtimestamp(item['created']).date() > end_date ]
    # Делим стенку на батчи 
    batches = [wall[i:i+25] for i in range(0,len(wall),25)]
    return batches 

# Поменяем post на photo
def make_url(this_uid, batch, token):
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token
    s = ''
    for bt in batch:
        s += 'API.likes.getList({"type":"photo","owner_id":"' + str(this_uid) + \
             '","item_id":"' + str(bt[0]) + '","count":"1000"}),'
    return begin + s[:-1] + end 


loises = get_likes(all_uids[570], dwd = dphd) # Меняем грязные стены на грязные фотки 
loises['2016-11-12 16:07:44']

[2850042, 4345637, 123854432, 341235898, 392924348]

In [194]:
photo_likes = { }
for uid in tqdm_notebook(all_uids):
    try:
        photo_likes[uid] = get_likes(uid, dwd = dphd)
    except:
        print('Возникли проблемы с ', uid)
        
print(len(photo_likes))


with open('vk_total_photo_likes_' + version, 'wb') as f:
    pickle.dump(photo_likes, f)

HBox(children=(IntProgress(value=0, max=706), HTML(value='')))

Возникли проблемы с  266510396
Возникли проблемы с  52144231
Возникли проблемы с  422131817
Возникли проблемы с  333801583
Возникли проблемы с  235595930
Возникли проблемы с  90380460
Возникли проблемы с  263618776
Возникли проблемы с  5058781
Возникли проблемы с  49289444
Возникли проблемы с  189333760
Возникли проблемы с  5611806
Возникли проблемы с  137859376
Возникли проблемы с  65581359
Возникли проблемы с  27742513
Возникли проблемы с  272562514
Возникли проблемы с  123857237
Возникли проблемы с  248191351
Возникли проблемы с  143817090
Возникли проблемы с  172437917
Возникли проблемы с  247298498
Возникли проблемы с  43481543
Возникли проблемы с  224768482
Возникли проблемы с  286251515
Возникли проблемы с  148396546
Возникли проблемы с  63482547
Возникли проблемы с  5188414
Возникли проблемы с  73743189
Возникли проблемы с  297810775
Возникли проблемы с  212257672
Возникли проблемы с  67128213
Возникли проблемы с  62432186
Возникли проблемы с  31996903
Возникли проблемы с  1058

В принципе, по скачиванию это почти всё. Нам не хватает на данный момент ровно двух вещей: постов из пабликов, на которые подписан иканам и лайков из этих пабликов. Всё это добро мы скачаем отдельно, когда займёмся исследованием палбликов.  

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 